### Set up

In [2]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.0 MB/s 
     |████████████████████████████████| 182 kB 51.1 MB/s 
     |████████████████████████████████| 7.6 MB 62.4 MB/s 


In [3]:
pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
pip install ray

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 57.4 MB 1.2 MB/s 
     |████████████████████████████████| 8.8 MB 64.6 MB/s 
     |████████████████████████████████| 468 kB 57.7 MB/s 


In [6]:
pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 2.2 MB/s 
     |████████████████████████████████| 1.3 MB 25.5 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=9af221d6ee552011a4a64b2600f1270a7f9f49dd9618c03569bc7a03f47b798a
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers


In [7]:
# SETUP ------------------------------------------------------------------------------------------------------------------
import json, zipfile, os, sys, itertools, subprocess, random, sklearn #, nltk
# subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'matplotlib'])
import numpy as np
from transformers import pipeline
import pandas as pd
# pd.options.display.max_columns = None
from pandas import json_normalize
from statistics import mean
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, make_scorer, f1_score, recall_score, roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.translate.bleu_score import sentence_bleu
import xgboost
import psutil
import ray
import time
from xgboost import XGBClassifier
from rouge import Rouge
from sentence_transformers import SentenceTransformer
lemmatizer = WordNetLemmatizer()


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
cd '/content/drive/Shared drives/CIS 530/FinalProject'

/content/drive/Shared drives/CIS 530/FinalProject


### Load and Clean

In [10]:
# Load and Clean ----------------------------------------------------------------------------------------------------------
def load(JSON):
    '''Loads the 4 most common subreddits in chunks and saves it as a CSV file'''
    total_df = None; first = True; counter = 0
    with pd.read_json(JSON,chunksize= 10**5, lines= True) as reader:
        for chunk in reader:
            #chunk = chunk.loc[chunk['subreddit'].isin(['AskReddit', 'leagueoflegends', 'AdviceAnimals', 'funny'])]
            chunk = chunk.loc[chunk['subreddit'].isin(['leagueoflegends','funny'])]
            counter += 1
            if first:
                first = False
                total_df = chunk
            else:
                pd.concat([total_df,chunk])
                print(counter)
            if counter % 10 == 0:
                total_df.to_csv("highest.csv" + str(counter))
                print("saved")

def clean(csv):
  data = pd.read_csv(csv)
  data = data.drop(columns = ['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'author', 'id', 'title', 'subreddit_id', 'content_len', 'summary_len'])
  # dropping nulls
  data = data.dropna()
  print(data.isnull().sum())
  # dropping duplicates
  data = data.drop_duplicates(keep = 'first')
  print(data.subreddit.value_counts(normalize=True))
  # encoding target variables 0-1
  data['target'] = np.where(data['subreddit'] == 'leagueoflegends', 0, 1)
  # saving to csv 
  data.to_csv("cleaned_data.csv",index=False)


# Split into Train, Dev, Testing ------------------------------------------------------------------------------------------
def split(reddit_data):
    '''Split data into train, dev, test'''
    train, validate, test = np.split( reddit_data.sample( frac=1, random_state=42 ), 
                                    [int( 0.8*len( reddit_data ) ), int( 0.9*len( reddit_data ) ) ] )
    return train, validate, test 

### Summarization

#### Simple Baseline

In [11]:
# Simple Summarization Method ---------------------------------------------------------------------------------------------
def transformer_summary(body):
    '''Uses the pre-built transformers model to create a summarization'''
    #print("BODY: ", body)
    summarizer = pipeline('summarization')
    transformer_summary = summarizer(body, max_length = 80, min_length=8, do_sample = False)
    return transformer_summary

#### Evaluate

In [12]:
# BLEU Evaluation Method ---------------------------------------------------------------------------------------------
def summary_evaluate_bleu(pred_summary, original):
    '''
    Evaluation metrics; 
    ROUGE-L, ROUGE-SU, ROUGE-n, BLEU...
    '''
    reference = [original.split()]
    candidate = pred_summary.split()
    unigram = sentence_bleu(reference, candidate, weights=(1, 0, 0, 0))
    bigram = sentence_bleu(reference, candidate, weights=(0, 1, 0, 0))
    trigram = sentence_bleu(reference, candidate, weights=(0, 0, 1, 0))
    fourgram = sentence_bleu(reference, candidate, weights=(0, 0, 0, 1))
    compiled_bleu = [unigram, bigram, trigram, fourgram]
    return compiled_bleu

In [13]:
# ROUGE Evaluation Method ---------------------------------------------------------------------------------------------
def summary_evaluate_rouge(pred_summary, original):
    '''
    Evaluation metrics; 
    ROUGE-L, ROUGE-SU, ROUGE-n, BLEU...
    '''
    rouge = Rouge()
    scores = rouge.get_scores(pred_summary, original)
    return scores

### Classification

#### Simple Baseline

In [14]:
def simple_baseline(CSV, league):
    '''Simple baseline to categorize the four subreddits
    Majority baseline: classify everything as league of legends'''
    data = CSV.dropna()
    # features = [ feat1, feat2, feat3 ]
    # X = data[features]
    y_true = data.target
    y_pred = data.assign(pred = league).pred

    accuracy = accuracy_score(y_true, y_pred)
    roc = roc_auc_score(y_true, y_pred, average = "weighted")
    f1     = f1_score(y_true, y_pred, average="weighted")
    recall = recall_score(y_true, y_pred, average="weighted")
    precision = precision_score(y_true, y_pred, average="weighted")

    return f1, recall, precision, roc, accuracy

#### Strong Baseline

In [15]:
def strong_baseline(data, model: str, feat1: str, embed: str):
    '''Strong baseline to categorize the two subreddits
    for now, we will be running this on the (1) RAW CONTENT, (2) ORIGINAL SUMMARY, and (3) SIMPLE SUMMARY
    Moving forward, we can run this on our more complex summarization method.
    SOURCE: https://github.com/britt-allen/classifying_reddit_posts/blob/master/notebooks/project_3_modeling.ipynb'''

    # features = [feat1, pred_summary]
    features = [feat1]
    X = data[features]
    y = data.target

    # Create Training and Testing Sets
    X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                        random_state=42,
                                                        test_size=0.20)
    # Split Testing into Dev and Test
    X_test, X_val, y_test, y_val = train_test_split(X_train, y_train, 
                                                        random_state=42,
                                                        test_size=0.5)
    cv_text  = CountVectorizer(stop_words='english', strip_accents = 'ascii', ngram_range=(1, 6), min_df=.03)
    sbert_model = SentenceTransformer('all-MiniLM-L6-v2')
    X_train_text_df = []
    X_val_text_df = []
    X_test_text_df = []

    if embed == 'count':
      # Prep train data
      X_train_text  = cv_text.fit_transform(X_train[feat1])
      X_train_text_df  = pd.DataFrame(X_train_text.todense(),  columns=[x+'_text'  for x in  cv_text.get_feature_names_out()])
      
      # Prep dev data
      X_val_text  = cv_text.transform(X_val[feat1])
      X_val_text_df   = pd.DataFrame(X_val_text.todense(),   columns=[x+'_text'  for x in  cv_text.get_feature_names_out()])
      
      # Prep test data
      X_test_text  = cv_text.transform(X_test[feat1])
      X_test_text_df   = pd.DataFrame(X_test_text.todense(),   columns=[x+'_text'  for x in  cv_text.get_feature_names_out()])
    elif embed == 'sbert':
      encoded_train = sbert_model.encode(list(X_train[feat1]))
      encoded_val =  sbert_model.encode(list(X_val[feat1]))
      encoded_test = sbert_model.encode(list(X_test[feat1]))

      X_train_text_df = pd.DataFrame(encoded_train)
      X_val_text_df = pd.DataFrame(encoded_val)
      X_test_text_df = pd.DataFrame(encoded_test)
    
    scoring = ['accuracy', 'f1_macro']

    if model == 'rf_1':
        rf_params = {
            'n_estimators': [15, 24, 30],         
            'criterion': ['gini'],
            'max_depth': [None, 5, 13, 21],       
            'bootstrap': [True, False],
            'min_samples_split': [5, 7, 15, 25],  
            #'max_features': [None, 'log2', 'auto', .10, .25, .50],
            'warm_start': [True],                 
            'random_state': [42]}
        A = GridSearchCV(RandomForestClassifier(),
                        rf_params,   cv = 4,
                        verbose = 1, n_jobs = -1,
                        refit = "accuracy",
                        scoring = 'accuracy' )
        F = GridSearchCV(RandomForestClassifier(),
                        rf_params,   cv = 4,
                        verbose = 1, n_jobs = -1,
                        refit = "accuracy",
                        scoring = 'f1_macro' )
        roc = GridSearchCV(RandomForestClassifier(),
                        rf_params,   cv = 4,
                        verbose = 1, n_jobs = -1,
                        refit = "accuracy",
                        scoring = 'roc_auc_ovr' )
    elif model == 'xgb':
      rf_params = {
        'n_estimators': [15, 30, 50, 100],
        'learning_rate': [0.01, 0.1, 0.3],
        'max_depth': [5, 10, 15],
        'colsample_bytree': [i/10.0 for i in range(1, 3)],
        'gamma': [i/10.0 for i in range(3)],
      }
      A = GridSearchCV(XGBClassifier(objective='binary:logistic'),
                        rf_params,   cv = 4,
                        verbose = 1, n_jobs = -1,
                        refit = "accuracy",
                        scoring = 'accuracy' )
      F = GridSearchCV(XGBClassifier(objective='binary:logistic'),
                        rf_params,   cv = 4,
                        verbose = 1, n_jobs = -1,
                        refit = "accuracy",
                        scoring = 'f1_macro' )
      roc = GridSearchCV(XGBClassifier(objective='binary:logistic'),
                        rf_params,   cv = 4,
                        verbose = 1, n_jobs = -1,
                        refit = "accuracy",
                        scoring = 'roc_auc_ovr' )
        
    # Fit the model on the training predictors and target
    A.fit(X_train_text_df.values, y_train)
    F.fit(X_train_text_df.values, y_train)
    roc.fit(X_train_text_df.values, y_train)

    # Predictions
    predictions_on_train = A.predict(X_train_text_df.values)
    predictions_on_val  = A.predict(X_val_text_df.values)
    predictions_on_test  = A.predict(X_test_text_df.values)

    # Acc Scores
    train_score_a = A.score(X_train_text_df.values, y_train)
    val_score_a   = A.score(X_val_text_df.values, y_val)
    test_score_a  = A.score(X_test_text_df.values, y_test)
    # F1 Scores
    train_score_f = F.score(X_train_text_df.values, y_train)
    val_score_f   = F.score(X_val_text_df.values, y_val)
    test_score_f  = F.score(X_test_text_df.values, y_test)
    # ROC scores
    train_score_r = roc.score(X_train_text_df.values, y_train)
    val_score_r   = roc.score(X_val_text_df.values, y_val)
    test_score_r  = roc.score(X_test_text_df.values, y_test)
    # Results
    results = A.cv_results_
    # Confusion Matrix
    confusion_matrix_train = confusion_matrix(y_train, predictions_on_train)
    confusion_matrix_test = confusion_matrix(y_test, predictions_on_test)
    best_params = A.best_params_

    d = {   'split '     : ['train',         'validation',  'test'],
            'acc '       : [train_score_a,        val_score_a,   test_score_a], 
            'F1 '        : [train_score_f, val_score_f, test_score_f],
            'roc_auc '   : [train_score_r, val_score_r, test_score_r], }
    table = pd.DataFrame(d)

    return A, F, roc, table, confusion_matrix_test, best_params, cv_text
    

#### Evaluate

In [16]:
import time
def evaluate_pred(A, F, roc, y, summary_pred, cv_text, embed):
    ''' Want to classify summaries given by transformer'''
    sbert_model = SentenceTransformer('all-MiniLM-L6-v2')
    
    start_time = time.time()
    summary_pred_text_df = []
    # Prep data
    if embed == 'count':
      summary_pred_text  = cv_text.transform(summary_pred)
      summary_pred_text_df  = pd.DataFrame(summary_pred_text.todense(),  columns=[x+'_text'  for x in  cv_text.get_feature_names_out()])
    elif embed == 'sbert':
      encoded_pred = sbert_model.encode(list(summary_pred))
      summary_pred_text_df = pd.DataFrame(encoded_pred)
      
    # Predictions
    predictions_on_a = A.predict(summary_pred_text_df.values)
    # Acc Scores
    score_a = A.score(summary_pred_text_df.values, y)
    print("Completed in  {} seconds".format(time.time()-start_time))
    # F1 Scores
    score_f = F.score(summary_pred_text_df.values, y)
    # ROC scores
    score_r = roc.score(summary_pred_text_df.values, y)
    
    # Results
    results = A.cv_results_
    # Confusion Matrix
    confusion = confusion_matrix(y, predictions_on_a)

    d = {   'acc '       : [score_a], 
            'F1 '        : [score_f],
            'roc_auc '   : [score_r], }
    table = pd.DataFrame(d)

    return table, confusion, predictions_on_a


### Main

In [17]:
if __name__ == "__main__":

    reddit_data = pd.read_csv("reddit_w_extr_summ.csv")
    ###### MILESTONE 2##########
    '''Training RF models and a majority classifier using content and the summaries and comparing their results'''

    print('--------Random Forest using content classification----------------')
    A, F, roc, table, confusion_matrix_test, best_params, cv_text = strong_baseline(
                                    reddit_data, "rf_1", "content", "count")

    print(table.T)
    print(best_params)

    print(' ')
    print('--------Random Forest using summary classification----------------')
    A_rfs, F_rfs, roc_rfs, table_rfs, confusion_matrix_test_rfs, best_params_rfs, cv_text_rfs = strong_baseline(
                                    reddit_data, "rf_1", "abstr_summary", "count")
    print(table_rfs.T) 
    print(best_params_rfs)

    print(' ')
    print('------- Simple Baseline Classification --------------')
    f1_baseline, recall_baseline, precision_baseline, roc, accuracy = simple_baseline(reddit_data, 0)
    print('f1 baseline: ', f1_baseline )
    print('recall baseline: ', recall_baseline )
    print('precision_baseline: ', precision_baseline )
    print('accuracy baseline: ', accuracy)
    print('roc baseline: ', roc)

--------Random Forest using content classification----------------


Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

Fitting 4 folds for each of 96 candidates, totalling 384 fits
Fitting 4 folds for each of 96 candidates, totalling 384 fits
Fitting 4 folds for each of 96 candidates, totalling 384 fits
                 0           1         2
split        train  validation      test
acc       0.921308    0.922518  0.920097
F1        0.920938    0.922362  0.919415
roc_auc   0.981146    0.982523  0.979812
{'bootstrap': False, 'criterion': 'gini', 'max_depth': 21, 'min_samples_split': 5, 'n_estimators': 30, 'random_state': 42, 'warm_start': True}
 
--------Random Forest using summary classification----------------
Fitting 4 folds for each of 96 candidates, totalling 384 fits
Fitting 4 folds for each of 96 candidates, totalling 384 fits
Fitting 4 folds for each of 96 candidates, totalling 384 fits
                 0           1         2
split        train  validation      test
acc       0.639528    0.642857  0.636199
F1        0.636093    0.638662  0.633452
roc_auc   0.697864    0.691116  0.705088
{'boot

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### Classification comparison 

##### Classification using Count Embeddings



In [18]:
reddit_data = pd.read_csv("reddit_w_extr_summ.csv")

In [19]:
('--------Random Forest using content classification & count embeddings----------------')
A_rfs, F_rfs, roc_rfs, table_rfs, confusion_matrix_test_rfs, best_params_rfs, cv_text_rfs = strong_baseline(
                                    reddit_data, "rf_1", "content", "count")
print(table_rfs.T) 
print(best_params_rfs)   

print('-------- Classification on extractive summary using RF & count embeddings----------------')
y = reddit_data.target
summary_pred = reddit_data.extr_summary
table2, confusion_mtrx, preds2 = evaluate_pred(A_rfs, F_rfs, roc_rfs, y, summary_pred, cv_text_rfs, "count")

print(table2.T)
print(confusion_mtrx)

print('-------- Classification on abstractive summary using RF & count embeddings ----------------')
summary_pred = reddit_data.abstr_summary	
table3, confusion_mtrx_3, preds3 = evaluate_pred(A_rfs, F_rfs, roc_rfs, y, summary_pred, cv_text_rfs, "count")

print(table3.T)
print(confusion_mtrx_3)

print('-------- Classification on all content using RF & count embeddings ----------------')
summary_pred = reddit_data.content	
table4, confusion_mtrx_4, preds4= evaluate_pred(A_rfs, F_rfs, roc_rfs, y, summary_pred, cv_text_rfs, "count")

print(table3.T)
print(confusion_mtrx_3)

Fitting 4 folds for each of 96 candidates, totalling 384 fits
Fitting 4 folds for each of 96 candidates, totalling 384 fits
Fitting 4 folds for each of 96 candidates, totalling 384 fits
                 0           1         2
split        train  validation      test
acc       0.921308    0.922518  0.920097
F1        0.920938    0.922362  0.919415
roc_auc   0.981146    0.982523  0.979812
{'bootstrap': False, 'criterion': 'gini', 'max_depth': 21, 'min_samples_split': 5, 'n_estimators': 30, 'random_state': 42, 'warm_start': True}
-------- Classification on extractive summary using RF & count embeddings----------------
Completed in  0.5787515640258789 seconds
                 0
acc       0.860324
F1        0.860145
roc_auc   0.926109
[[1851  509]
 [  68 1703]]
-------- Classification on abstractive summary using RF & count embeddings ----------------
Completed in  0.2839851379394531 seconds
                 0
acc       0.705640
F1        0.699988
roc_auc   0.796533
[[1174 1186]
 [  30 174

In [ ]:
print('--------XGB using content classification & count embeddings----------------')
A_xgb, F_xgb, roc_xgb, table_xgb, confusion_matrix_xgb, best_params_xgb, cv_text_xgb = strong_baseline(
                                    reddit_data, "xgb", "content", "count")
    
print(table_xgb.T) 
print(best_params_xgb)

print('-------- Classification on transformer summary using XGB & count embeddings----------------')
y = reddit_data.target
summary_pred = reddit_data.extr_summary
table7, confusion_mtrx_7, preds7 = evaluate_pred(A_xgb, F_xgb, roc_xgb, y, summary_pred, cv_text_xgb, "count")

print(table7.T)
print(confusion_mtrx_7)

print('-------- Classification on abstractive summary using XGB & count embeddings----------------')
summary_pred = reddit_data.abstr_summary	
table5, confusion_mtrx_5, preds5 = evaluate_pred(A_xgb, F_xgb, roc_xgb, y, summary_pred, cv_text_xgb, "count")

print(table5.T)
print(confusion_mtrx_5)

print('-------- Classification on all content using XGB & count embeddings ----------------')
summary_pred = reddit_data.content
table6, confusion_mtrx_6, preds6 = evaluate_pred(A_xgb, F_xgb, roc_xgb, y, summary_pred, cv_text_xgb, "count")

print(table6.T)
print(confusion_mtrx_6)

--------XGB using content classification----------------
Fitting 4 folds for each of 216 candidates, totalling 864 fits
Fitting 4 folds for each of 216 candidates, totalling 864 fits
Fitting 4 folds for each of 216 candidates, totalling 864 fits
                 0           1         2
split        train  validation      test
acc       0.918281    0.918281  0.918281
F1        0.917756    0.917998  0.917437
roc_auc   0.980572    0.981922  0.979294
{'colsample_bytree': 0.1, 'gamma': 0.0, 'learning_rate': 0.3, 'max_depth': 5, 'n_estimators': 100}
-------- Classification on transformer summary using XGB ----------------
Completed in  0.5508489608764648 seconds
                 0
acc       0.863713
F1        0.863360
roc_auc   0.931801
[[1889  471]
 [  92 1679]]
-------- Classification on abstractive summary using XGB ----------------
Completed in  0.25338125228881836 seconds
                 0
acc       0.717986
F1        0.713995
roc_auc   0.803884
[[1239 1121]
 [  44 1727]]
-------- Clas

##### Classification using S-BERT Embeddings

In [20]:
('--------Random Forest using content classification & count embeddings----------------')
A_rfs, F_rfs, roc_rfs, table_rfs, confusion_matrix_test_rfs, best_params_rfs, cv_text_rfs = strong_baseline(
                                    reddit_data, "rf_1", "content", "sbert")
print(table_rfs.T) 
print(best_params_rfs)   

print('-------- Classification on extractive summary using RF & sbert embeddings----------------')
y = reddit_data.target
summary_pred = reddit_data.extr_summary
table2, confusion_mtrx, preds2 = evaluate_pred(A_rfs, F_rfs, roc_rfs, y, summary_pred, cv_text_rfs, "sbert")

print(table2.T)
print(confusion_mtrx)

print('-------- Classification on abstractive summary using RF & sbert embeddings ----------------')
summary_pred = reddit_data.abstr_summary	
table3, confusion_mtrx_3, preds3 = evaluate_pred(A_rfs, F_rfs, roc_rfs, y, summary_pred, cv_text_rfs, "sbert")

print(table3.T)
print(confusion_mtrx_3)

print('-------- Classification on all content using RF & sbert embeddings ----------------')
summary_pred = reddit_data.content	
table4, confusion_mtrx_4, preds4= evaluate_pred(A_rfs, F_rfs, roc_rfs, y, summary_pred, cv_text_rfs, "sbert")

print(table3.T)
print(confusion_mtrx_3)

Fitting 4 folds for each of 96 candidates, totalling 384 fits
Fitting 4 folds for each of 96 candidates, totalling 384 fits
Fitting 4 folds for each of 96 candidates, totalling 384 fits
                 0           1         2
split        train  validation      test
acc            1.0         1.0       1.0
F1        0.996916    0.998159  0.995658
roc_auc   0.999801     0.99989  0.999707
{'bootstrap': False, 'criterion': 'gini', 'max_depth': None, 'min_samples_split': 7, 'n_estimators': 30, 'random_state': 42, 'warm_start': True}
-------- Classification on extractive summary using RF & sbert embeddings----------------
Completed in  82.73971438407898 seconds
                 0
acc       0.944323
F1        0.934652
roc_auc   0.977734
[[2170  190]
 [  40 1731]]
-------- Classification on abstractive summary using RF & sbert embeddings ----------------
Completed in  38.97196936607361 seconds
                 0
acc       0.825708
F1        0.823960
roc_auc   0.909372
[[1734  626]
 [  94 167

In [21]:
print('--------XGB using content classification & sbert embeddings----------------')
A_xgb, F_xgb, roc_xgb, table_xgb, confusion_matrix_xgb, best_params_xgb, cv_text_xgb = strong_baseline(
                                    reddit_data, "xgb", "content", "sbert")
    
print(table_xgb.T) 
print(best_params_xgb)

print('-------- Classification on transformer summary using XGB & sbert embeddings----------------')
y = reddit_data.target
summary_pred = reddit_data.extr_summary
table7, confusion_mtrx_7, preds7 = evaluate_pred(A_xgb, F_xgb, roc_xgb, y, summary_pred, cv_text_xgb, "sbert")

print(table7.T)
print(confusion_mtrx_7)

print('-------- Classification on abstractive summary using XGB & sbert embeddings----------------')
summary_pred = reddit_data.abstr_summary	
table5, confusion_mtrx_5, preds5 = evaluate_pred(A_xgb, F_xgb, roc_xgb, y, summary_pred, cv_text_xgb, "sbert")

print(table5.T)
print(confusion_mtrx_5)

print('-------- Classification on all content using XGB & sbert embeddings ----------------')
summary_pred = reddit_data.content
table6, confusion_mtrx_6, preds6 = evaluate_pred(A_xgb, F_xgb, roc_xgb, y, summary_pred, cv_text_xgb, "sbert")

print(table6.T)
print(confusion_mtrx_6)

--------XGB using content classification & sbert embeddings----------------
Fitting 4 folds for each of 216 candidates, totalling 864 fits
Fitting 4 folds for each of 216 candidates, totalling 864 fits
Fitting 4 folds for each of 216 candidates, totalling 864 fits
              0           1     2
split     train  validation  test
acc         1.0         1.0   1.0
F1          1.0         1.0   1.0
roc_auc     1.0         1.0   1.0
{'colsample_bytree': 0.1, 'gamma': 0.0, 'learning_rate': 0.1, 'max_depth': 15, 'n_estimators': 100}
-------- Classification on transformer summary using XGB & sbert embeddings----------------
Completed in  150.72877478599548 seconds
                 0
acc       0.950133
F1        0.949579
roc_auc   0.986862
[[2179  181]
 [  25 1746]]
-------- Classification on abstractive summary using XGB & sbert embeddings----------------
Completed in  69.53825116157532 seconds
                 0
acc       0.844106
F1        0.843929
roc_auc   0.929430
[[1813  547]
 [  97 1

In [22]:
confusion_matrix_xgb

array([[957,   0],
       [  0, 695]])

In [ ]:
print('--------Caclulating transformer summaries in parallel----------------')

pipe = pipeline("summarization",batch_size=1, device=-1)
    
num_cpus = psutil.cpu_count(logical=True)
print('Number of available CPUs:', num_cpus)

# Start Ray cluster
ray.init(num_cpus=9, ignore_reinit_error=True)

"""
The command ray.put(x) would be run by a worker process or by the driver process (the driver process is the one running your script). 
It takes a Python object and copies it to the local object store (here local means on the same node). 
Once the object has been stored in the object store, its value cannot be changed.
In addition, ray.put(x) returns an object ID, which is essentially an ID that can be used to refer to the newly created remote object. 
If we save the object ID in a variable with x_id = ray.put(x), 
then we can pass x_id into remote functions, 
and those remote functions will operate on the corresponding remote object.
"""
pipe_id = ray.put(pipe)

# @ray.remote decorator enables to use this 
# function in distributed setting
@ray.remote
def predict(pipeline, text_data):
    return pipeline(text_data, max_length = 1600, min_length = 8, do_sample = False)

# Predict multipe texts on all available CPUs and time the inference duration
start = time.time()

# Run the function using multiple cores and gather the results
predictions = ray.get([predict.remote(pipe_id, text) for text in reddit_data.content.values[:200]])
predictions = [x[0]["summary_text"] for x in predictions]
end = time.time()
pd.DataFrame(predictions).to_csv("alex.csv", header = False)
# Stop running Ray cluster
ray.shutdown()

# Evaluation Scripts

In [ ]:
# create and evaluate transformer summaries 

reddit_data = pd.read_csv("cleaned_data.csv")
pred = []
bleu_1gram = []
bleu_2gram = []
bleu_3gram = []
bleu_4gram = []
rouge_1 = []
rouge_2 = []
rouge_l = []
i = 0
for content in reddit_data.content.values:
  print(i)

  pred_summary = transformer_summary(content)
  original_summary = reddit_data.summary.values[i]

  pred_summary = pred_summary[0]['summary_text']
  bleu_scores = summary_evaluate_bleu(pred_summary, original_summary)
  rouge_scores = summary_evaluate_rouge(pred_summary, original_summary)

  pred.append(pred_summary)
  bleu_1gram.append(bleu_scores[0])
  bleu_2gram.append(bleu_scores[1])
  bleu_3gram.append(bleu_scores[2])
  bleu_4gram.append(bleu_scores[3])
  rouge_1.append(rouge_scores[0]['rouge-1'])
  rouge_2.append(rouge_scores[0]['rouge-2'])
  rouge_l.append(rouge_scores[0]['rouge-l'])

  i+=1

reddit_data['pred_summary'] = pred
reddit_data['bleu_1gram'] = bleu_1gram
reddit_data['bleu_2gram'] = bleu_2gram
reddit_data['bleu_3gram'] = bleu_3gram
reddit_data['bleu_4gram'] = bleu_4gram
reddit_data['rouge_1'] = rouge_1
reddit_data['rouge_2'] = rouge_2
reddit_data['rouge_l'] = rouge_l

In [ ]:
# evaluate the existing calculated transformer summaries vs original summaries 

transformer_summaries = pd.read_csv("reddit_w_extr_summ.csv")
transformer_summaries_scores = pd.DataFrame()

pred = []
original = []
bleu_1gram = []
bleu_2gram = []
bleu_3gram = []
bleu_4gram = []
rouge_1 = []
rouge_2 = []
rouge_l = []
i = 0
for abstr_summary in transformer_summaries.abstr_summary.values:

  pred_summary = transformer_summaries.extr_summary[i]
  original_summary = abstr_summary

  bleu_scores = summary_evaluate_bleu(pred_summary, original_summary)
  rouge_scores = summary_evaluate_rouge(pred_summary, original_summary)

  pred.append(pred_summary)
  original.append(original_summary)
  bleu_1gram.append(bleu_scores[0])
  bleu_2gram.append(bleu_scores[1])
  bleu_3gram.append(bleu_scores[2])
  bleu_4gram.append(bleu_scores[3])
  rouge_1.append(rouge_scores[0]['rouge-1'])
  rouge_2.append(rouge_scores[0]['rouge-2'])
  rouge_l.append(rouge_scores[0]['rouge-l'])

  i+=1
  
transformer_summaries_scores['pred_summary'] = pred
transformer_summaries_scores['original_summary'] = original
transformer_summaries_scores['bleu_1gram'] = bleu_1gram
transformer_summaries_scores['bleu_2gram'] = bleu_2gram
transformer_summaries_scores['bleu_3gram'] = bleu_3gram
transformer_summaries_scores['bleu_4gram'] = bleu_4gram
transformer_summaries_scores['rouge_1'] = rouge_1
transformer_summaries_scores['rouge_2'] = rouge_2
transformer_summaries_scores['rouge_l'] = rouge_l

transformer_summaries_scores.to_csv("transformer_summaries_scores_full.csv")


In [ ]:
# evaluate the existing calculated transformer summaries vs raw text

transformer_summaries = pd.read_csv("reddit_w_extr_summ.csv")
transformer_summaries_raw_text_scores = pd.DataFrame()

pred = []
raw = []
bleu_1gram = []
bleu_2gram = []
bleu_3gram = []
bleu_4gram = []
rouge_1 = []
rouge_2 = []
rouge_l = []
i = 0
for normalizedBody in transformer_summaries.normalizedBody.values:

  pred_summary =  transformer_summaries.extr_summary[i]
  raw_text = normalizedBody

  bleu_scores = summary_evaluate_bleu(pred_summary, raw_text)
  rouge_scores = summary_evaluate_rouge(pred_summary, raw_text)

  pred.append(pred_summary)
  raw.append(raw_text)
  bleu_1gram.append(bleu_scores[0])
  bleu_2gram.append(bleu_scores[1])
  bleu_3gram.append(bleu_scores[2])
  bleu_4gram.append(bleu_scores[3])
  rouge_1.append(rouge_scores[0]['rouge-1'])
  rouge_2.append(rouge_scores[0]['rouge-2'])
  rouge_l.append(rouge_scores[0]['rouge-l'])

  i+=1
  
transformer_summaries_raw_text_scores['pred_summary'] = pred
transformer_summaries_raw_text_scores['raw_text'] = raw
transformer_summaries_raw_text_scores['bleu_1gram'] = bleu_1gram
transformer_summaries_raw_text_scores['bleu_2gram'] = bleu_2gram
transformer_summaries_raw_text_scores['bleu_3gram'] = bleu_3gram
transformer_summaries_raw_text_scores['bleu_4gram'] = bleu_4gram
transformer_summaries_raw_text_scores['rouge_1'] = rouge_1
transformer_summaries_raw_text_scores['rouge_2'] = rouge_2
transformer_summaries_raw_text_scores['rouge_l'] = rouge_l

transformer_summaries_raw_text_scores.to_csv("transformer_summaries_raw_text_scores_full.csv")


In [ ]:
# evaluate the original summaries vs raw text

original_summaries_raw_text_scores = pd.DataFrame()
reddit_data = pd.read_csv("reddit_w_extr_summ.csv")

original = []
raw = []
bleu_1gram = []
bleu_2gram = []
bleu_3gram = []
bleu_4gram = []
rouge_1 = []
rouge_2 = []
rouge_l = []
i = 0
for normalizedBody in reddit_data.normalizedBody.values:

  original_summary = reddit_data.abstr_summary.values[i]
  raw_text = normalizedBody

  bleu_scores = summary_evaluate_bleu(original_summary, raw_text)
  rouge_scores = summary_evaluate_rouge(original_summary, raw_text)

  original.append(original_summary)
  raw.append(raw_text)
  bleu_1gram.append(bleu_scores[0])
  bleu_2gram.append(bleu_scores[1])
  bleu_3gram.append(bleu_scores[2])
  bleu_4gram.append(bleu_scores[3])
  rouge_1.append(rouge_scores[0]['rouge-1'])
  rouge_2.append(rouge_scores[0]['rouge-2'])
  rouge_l.append(rouge_scores[0]['rouge-l'])

  i+=1
  
original_summaries_raw_text_scores['original_summary'] = original
original_summaries_raw_text_scores['raw_text'] = raw
original_summaries_raw_text_scores['bleu_1gram'] = bleu_1gram
original_summaries_raw_text_scores['bleu_2gram'] = bleu_2gram
original_summaries_raw_text_scores['bleu_3gram'] = bleu_3gram
original_summaries_raw_text_scores['bleu_4gram'] = bleu_4gram
original_summaries_raw_text_scores['rouge_1'] = rouge_1
original_summaries_raw_text_scores['rouge_2'] = rouge_2
original_summaries_raw_text_scores['rouge_l'] = rouge_l

original_summaries_raw_text_scores.to_csv("original_summaries_raw_text_scores_full.csv")


# Descriptive Statistics

In [ ]:
def create_confusion_matrix(confusion_matrix_test):
  ax = sns.heatmap(confusion_matrix_test/np.sum(confusion_matrix_test), annot=True, fmt='.2%', cmap='Blues')
  ax.set_title('Seaborn Confusion Matrix with labels\n\n');
  ax.set_xlabel('\nPredicted Values')
  ax.set_ylabel('Actual Values ');

  ## Ticket labels - List must be in alphabetical order
  ax.xaxis.set_ticklabels(['funny','leagueoflegends'])
  ax.yaxis.set_ticklabels(['funny','leagueoflegends'])

  plt.savefig("./confusion_matrix_test.png")


In [ ]:
def create_chart():
  data = pd.read_csv("cleaned_data.csv")
  data.subreddit.value_counts().plot(kind='barh')
  plt.title('Number of Observations by Subreddit')
  plt.xlabel('Number of Obersvations')
  plt.ylabel('Subreddit')
  plt.savefig("./num_of_observations.png")

create_chart()

# Error Analysis for Milestone 4

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
reddit_data.iloc[[149, 243, 289, 3613, 3779]][['content', 'extr_summary', 'subreddit', 'target']]

,content,extr_summary,subreddit,target
149,"Alternate link with full description - \n All those who play must sign a silly, yet binding contract that involves their participation for the rest of their entire lives! Check it out on Reddit and please tell me what you think. \n Bill Murray Tag is a game I invented for my friends and I to play in the month of February. It's fun, creative and is guaranteed to create memories for the rest of our lives. Our lives...wherever life takes us, we will be able to stay in touch, because we are forever members of the Bill Murray Tag Club. Enjoy and start your own BMT Club with your friends. \n This game was inspired by a Wall Street Journal article about friends who have been playing the same game of tag for decades. Read about it here -","Bill Murray Tag is a game I invented for my friends and I to play in the month of February . All those who play must sign a silly, yet binding contract that involves their participation for the rest of their entire lives . Enjoy and start your own BMT Club with your friends .",funny,1
243,"As lbr218 said, it would in fact be Vandy. The numbers don't lie, in that the state of Tennessee doesn't have the population to support two such fanbases (joke). \n The big difference here is how the school is perceived by the general population. The rest of the SEC schools are seen as sports teams rather than universitites by a larger portion of the local and national population. This is largely due to marketing decisions made in the last few decades to build up these schools athletic reputations, as evidenced by the SECs relatively recent rise to near complete football domination. Vanderbilt is lucky in that they benefit monetarily from the SEC's TV contract and the recruiting advantages of being in the SEC, but haven't seen the same level of success of the rest of the SEC schools due to their lesser brand recognition. So effectively, Vandy is different because they don't win, haha. \n Unforunately, what this boils down to is that if a team enjoys prolonged success, they tend to gather a larger and larger group of trashy fans. This is not to say that all fans of a school fit this mold, in fact, I've met many a classy fan from most schools that I would slap this label on.\nHating the SEC is like hating the Miami Heat, the Lakers, the Patriots, or the Yankees.",The rest of the SEC schools are seen as sports teams rather than universitites by a larger portion of the local and national population . This is largely due to marketing decisions made in the last few decades to build up these schools athletic reputations .,funny,1
289,"Basically, if Portugal beat Ghana, they would be at 4 points, tied with USA, it would go to goal differential, which would then be determined on the scores of the USA v Germany and Ghana v Portugal games. \n Let's say USA loses to Germany 1-3 (-2 GD to the current +1 so they are at 1GD in the end) and Portugal beats Ghana 4-0 (+4GD to current of -4 so they are at 0GD at the end). USA still moves on. \n Now change that to Ghana beats Portuagal 4-0. They are at -1 right now, so add in +4 and they are at +3GD final to 1GD of the US. Ghana moves on and USA is third. \n So best case scenario, we win vs Germany or tie and we're through. Worst case that we still get through would be we lose to Germany, but without a large goal differential so that Ghana would have a bit of a differential to make up, and Portugal would have to absolutely stomp Ghana if they wanted through.",Ghana beat Portugal 4-0 in Portugal's 4-1 win over Ghana . Goal differential would then be determined on the scores of USA v Germany and Ghana v Portugal games . USA still moves on and Ghana moves on .,funny,1
3613,"Yeah both my Fedex and UPS drivers are fuckin amazing. I've never once had an issue with delivery. Of course, I live in a quiet, middle-upper class suburb. Sorry, city folk :( \n The only issue I ever had was when I opened the door to a UPS guy needing a signature, and my d

In [ ]:
a = pd.DataFrame({'target': y, 'predicted': preds2})
one = a[a['target']== 1]
one[one['target']!= one['predicted']]

,target,predicted
149,1,0
243,1,0
289,1,0
366,1,0
389,1,0
...,...,...
3613,1,0
3624,1,0
3779,1,0
3785,1,0


In [ ]:
confusion_mtrx_7

NameError: ignored

In [ ]:
confusion_mtrx_5

NameError: ignored